# Playground

In [1]:
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

True

In [2]:
# Hide INFO logs regarding token usage, etc
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

## Setup

### Generate some example Documents

In [3]:
from llama_index import download_loader
from llama_index import (
    GPTVectorStoreIndex as VectorStoreIndex,
    GPTTreeIndex as TreeIndex
)

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()
documents = loader.load_data(pages=['Berlin'])

### Create a list of any sort of indices (custom LLMs, custom embeddings, etc)

In [4]:
indices = [VectorStoreIndex.from_documents(documents), TreeIndex.from_documents(documents)]

## Using the Playground


### Initialize with indices

In [5]:
from llama_index.playground import Playground

playground = Playground(indices=indices)

In [6]:
playground.compare("What is the population of Berlin?")

Query:
What is the population of Berlin?

GPTVectorStoreIndex, retriever mode = default

The population of Berlin is 3.7 million.

GPTTreeIndex, retriever mode = select_leaf

It is not possible to answer this question with the given context information.

GPTTreeIndex, retriever mode = select_leaf_embedding

The population of Berlin is approximately 3.7 million inhabitants.

GPTTreeIndex, retriever mode = all_leaf


The population of Berlin is estimated to be around 3.7 million inhabitants, making it the most populous city in Germany. It is home to a diverse population, with people from over 190 countries living in the city. The largest religious denomination recorded in 2018 was the Protestant regional church body—the Evangelical Church of Berlin-Brandenburg-Silesian Upper Lusatia (EKBO)—a united church. EKBO is a member of the Evangelical Church in Germany (EKD) and Union Evangelischer Kirchen (UEK). According to the EKBO, their membership accounted for 18.7 percent of the local popul

,Index,Retriever Mode,Output,Duration,LLM Tokens,Embedding Tokens
0,GPTVectorStoreIndex,default,\nThe population of Berlin is 3.7 million.,2.776464,1799,0
1,GPTTreeIndex,select_leaf,\nIt is not possible to answer this question w...,16.467436,896,0
2,GPTTreeIndex,select_leaf_embedding,\nThe population of Berlin is approximately 3....,10.731784,913,0
3,GPTTreeIndex,all_leaf,\n\nThe population of Berlin is estimated to b...,733.014671,38632,0
4,GPTTreeIndex,root,\nThe population of Berlin is 3.7 million with...,3.582931,696,0


### Initialize with Documents

In [ ]:
# Uses documents in a preset list of indices
playground = Playground.from_docs(documents=documents)

### Save the indices for future use

In [24]:
tree_index_persist_dir = '/media/sunhuawei/data/files/gpt_tree_index/wikipedia/berlin'
vector_index_persist_dir = '/media/sunhuawei/data/files/gpt_vector_store_index/wikipedia/berlin'

In [11]:
indices[0].storage_context.persist(persist_dir=vector_index_persist_dir)
indices[1].storage_context.persist(persist_dir=tree_index_persist_dir)

### Reload from disk

In [12]:
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir=tree_index_persist_dir)

# load index
tree_index = load_index_from_storage(storage_context)

### Query the index

In [44]:
query_engine = tree_index.as_query_engine(child_branch_factor=2)
response = query_engine.query('What is the population of Berlin?')
print(response)
print(response.get_formatted_sources())


The population of Berlin is 3.7 million.
> Source (Doc id: d13ab134-ce57-4db4-8339-c6695874e917): Berlin ( bur-LIN, German: [bɛʁˈliːn] (listen)) is the capital and largest city of Germany by both...

> Source (Doc id: 630cf07e-5c52-4ee3-ac56-601b37992237): western part, underlining the US support for West Berlin. Berlin was completely divided. Although...


In [16]:
from llama_index import ResponseSynthesizer
from llama_index.indices.postprocessor import KeywordNodePostprocessor, PrevNextNodePostprocessor, SimilarityPostprocessor
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import VectorIndexRetriever

In [17]:
storage_cxt = StorageContext.from_defaults(persist_dir=vector_index_persist_dir)
vector_index = load_index_from_storage(storage_cxt)
query_engine = vector_index.as_query_engine()
response = query_engine.query('What is the population of Berlin?')
print(response)


The population of Berlin is 3.7 million.


In [53]:
from llama_index import SimpleDirectoryReader
filename_fn = lambda filename: {'file_name': filename}

# automatically sets the extra_info of each document according to filename_fn
documents = SimpleDirectoryReader(vector_index_persist_dir, file_metadata=filename_fn).load_data()

In [54]:
documents[0].extra_info

{'file_name': '/media/sunhuawei/data/files/gpt_vector_store_index/wikipedia/berlin/docstore.json'}

In [55]:
documents[0].extra_info_str

'file_name: /media/sunhuawei/data/files/gpt_vector_store_index/wikipedia/berlin/docstore.json'

In [56]:
documents[0].doc_id

'6cecb12c-a5b1-4787-9633-a350b2aa0617'

In [47]:
SimpleDirectoryReader(vector_index_persist_dir, filename_as_id=True)

TypeError: SimpleDirectoryReader.__init__() got an unexpected keyword argument 'filename_as_id'

In [48]:
print(tree_index.ref_doc_info)

{'1018c7a6-d4b9-4b16-9467-2041497b2ce1': RefDocInfo(doc_ids=['d13ab134-ce57-4db4-8339-c6695874e917', '2ac89c0c-7433-4801-beb5-d88d05c0c2b4', '10a62926-6e44-454a-9912-ff3fdb4e9a31', 'fcbdc860-a489-40bd-8fdd-3f3e60f22125', '630cf07e-5c52-4ee3-ac56-601b37992237', '87e62ed9-8e8f-4ce6-8928-63cfa401e9b2', 'e277eff6-30e8-4863-9b51-d2aefca42f79', 'ad9d0b74-f33a-40b8-bf93-05dd9ad1170c', '7b5d9bd0-68cb-4239-86a3-cca0be85008f', '12cbfdf7-1c1a-442b-b8bb-40de91db2dad', 'df15e3d2-48f1-4f6d-9ea0-51d98188c6a5', '30839cf8-1953-4c3a-85bb-53ff9ed45c46', '2f715cab-a245-452b-b86c-de2f31d91ddb', '15950c33-ad7c-4e4e-9a3c-2afbd536a2fb', 'ca1a3a9f-ea8e-4463-9dcd-2b45780a9450', '00a149f2-a0db-491e-a0e0-4b0061c8c577', '4b6decb4-8844-42ae-b14d-4bcbc515c14b', '1abe7f84-5d39-45dd-bd7e-32a6d34167dd', '62511a5b-29d6-441f-99f8-c49994991937', '11413520-6ff1-4bcc-bcce-41a298cf40fc', '6c8b47c4-ff0c-40a5-96d9-0399e113b5ab'], extra_info={})}
